# Housing Market Analysis: London Case

Description: Historical prices, sales, crimes, resident satisfaction and salaries by borough.

## Content

[The datasets](https://www.kaggle.com/datasets/justinas/housing-in-london) is primarily centered around the housing market of London. However, it contains a lot of additional relevant data:

- Monthly average house prices
- Yearly number of houses
- Yearly number of houses sold
- Yearly percentage of households that recycle
- Yearly life satisfaction
- Yearly median salary of the residents of the area
- Yearly mean salary of the residents of the area
- Monthly number of crimes committed
- Yearly number of jobs
- Yearly number of people living in the area
- Area size in hectares

The data is split by areas of London called boroughs (a flag exists to identify these), but some of the variables have other geographical UK regions for reference (like England, North East, etc.). There have been no changes made to the data except for melting it into a long format from the original tables.

#### DataFrames
- df: main dataframe
- london_df: london boroughs
- region_df: regions of UK

## Package Installation

In [ ]:
!pip install dtale

## Importing Libraries

In [2]:
import pandas as pd
import dtale
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Read Dataset

In [31]:
df = pd.read_csv("housing_in_london_yearly_variables.csv")

## View & Analyze

In [32]:
dtale.show(df)

/Users/iremkurt/anaconda3/lib/python3.11/site-packages/dtale/views.py:778: FutureWarning:

['recycling_pct'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.



## Data Cleaning

In [33]:
# Count the number of missing rows
df.isna().sum()

code                   0
area                   0
date                   0
median_salary         22
life_satisfaction    719
mean_salary            0
recycling_pct        211
population_size       53
number_of_jobs       140
area_size            405
no_of_houses         405
borough_flag           0
dtype: int64

### Median Salary

In [34]:
# Calculate the mean of "median_salary" column
median_salary_mean = df["median_salary"].mean()

# Fill null values with the mean
df["median_salary"].fillna(median_salary_mean, inplace=True)

# Calculate the total number of null values in "median_salary" column
median_salary_null_count = df["median_salary"].isnull().sum()

### Life Satisfaction

In [35]:
# Calculate the total number of null values in "life_satisfaction" column
life_satisfaction_null_count = df["life_satisfaction"].isnull().sum()

# Fill NaN values with random values between 6.5 and 7 because the value is 6.8 in OECD Better Life Index
df['life_satisfaction'].fillna(pd.Series(np.random.uniform(6.5, 7, size=len(df))), inplace=True)

### Mean Salary

In [8]:
# Convert non-numeric values to NaN in the 'mean_salary' column
df['mean_salary'] = pd.to_numeric(df['mean_salary'], errors='coerce')

# Convert the 'mean_salary' column to float data type
df['mean_salary'] = df['mean_salary'].astype(float)

# Check the new data type of the 'mean_salary' column
print("New data type of 'mean_salary' column:", df['mean_salary'].dtype)

New data type of 'mean_salary' column: float64


In [9]:
# Calculate the total number of null values in "mean_salary" column
mean_salary_null_count = df["mean_salary"].isnull().sum()

# Calculate the mean of "mean_salary" column
mean_salary_mean = df["mean_salary"].mean()

# Fill null values with the mean
df["mean_salary"].fillna(mean_salary_mean, inplace=True)

In [10]:
# Get all distinct values from the 'area' column
distinct_area_names = df['area'].unique()
distinct_area_names

array(['city of london', 'barking and dagenham', 'barnet', 'bexley',
       'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield',
       'greenwich', 'hackney', 'hammersmith and fulham', 'haringey',
       'harrow', 'havering', 'hillingdon', 'hounslow', 'islington',
       'kensington and chelsea', 'kingston upon thames', 'lambeth',
       'lewisham', 'merton', 'newham', 'redbridge',
       'richmond upon thames', 'southwark', 'sutton', 'tower hamlets',
       'waltham forest', 'wandsworth', 'westminster', 'north east',
       'north west', 'yorkshire and the humber', 'east midlands',
       'west midlands', 'east', 'london', 'south east', 'south west',
       'inner london', 'outer london', 'england', 'united kingdom',
       'great britain', 'england and wales', 'northern ireland',
       'scotland', 'wales'], dtype=object)

In [11]:
# Concatenate 'region_df' and 'london_df' into a single DataFrame
# combined_df = pd.concat([region_df, london_df], ignore_index=True)

# Find the rows in 'df' that are not in 'combined_df'
# different_rows = df[~df['area'].isin(combined_df['area'])]

# Get distinct values in the 'area' column of 'different_rows'
# distinct_areas_in_different_rows = different_rows['area'].unique()
# distinct_areas_in_different_rows

### Recycling Percentage

In [12]:
# Convert the "recycling_pct" column to numeric type
df["recycling_pct"] = pd.to_numeric(df["recycling_pct"], errors="coerce")

# Print the updated data type of the column
df["recycling_pct"].dtypes

# Group the DataFrame by "area" and calculate the mean recycling percentage for each area
mean_recycling_pct = df.groupby("area")["recycling_pct"].mean()
mean_recycling_pct

# Convert Series to DataFrame
mean_recycling_pct = mean_recycling_pct.reset_index()
mean_recycling_pct

# Merging DataFrames based on the 'city' column (common key)
df = df.merge(mean_recycling_pct, on='area', how='left')

In [13]:
# Fill NaN values in "recycling_pct_x" with values from "recycling_pct_y"
df['recycling_pct_x'] = df['recycling_pct_x'].fillna(df['recycling_pct_y'])

# Calculate the median of "recycling_pct_x" column
recycling_pct_x_median = df['recycling_pct_x'].median()

# Fill NaN values in "recycling_pct_x" column with the median value
df['recycling_pct_x'] = df['recycling_pct_x'].fillna(recycling_pct_x_median)

### Population Size

In [14]:
# Getting population size data by year
pop_size = pd.read_csv("population_size.csv")

# List of columns to drop
columns_to_drop = ["sample", "2001", "2011", "2021", "Data.1"]

# Drop the specified columns from the DataFrame
pop_size.drop(columns=columns_to_drop, inplace=True)

In [16]:
# Merging DataFrames based on the 'city' column (common key)
df = df.merge(pop_size, on='area', how='left')

# Rename the columns as required
df.rename(columns={
    "recycling_pct_x": "recycling_pct",
    "recycling_pct_y": "mean_recycle_pct",
    "Data": "extended_pop_size"
}, inplace=True)

In [18]:
# Identify rows where "population_size" is NaN
mask = df['population_size'].isnull()

# Fill NaN values in "population_size" column with corresponding values from "extended_pop_size"
df.loc[mask, 'population_size'] = df.loc[mask, 'extended_pop_size']

In [ ]:
mask = df['population_size'].isnull()

In [28]:
pop_size

,area,Data
0,barking and dagenham,"190,533"
1,barnet,"355,461"
2,bexley,"232,682"
3,brent,"307,220"
4,bromley,"312,248"
5,camden,"210,941"
6,city of westminster,"209,039"
7,croydon,"363,545"
8,ealing,"337,910"
9,enfield,"307,073"


In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1071 entries, 0 to 1070
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   code               1071 non-null   object 
 1   area               1071 non-null   object 
 2   date               1071 non-null   object 
 3   median_salary      1071 non-null   float64
 4   life_satisfaction  1071 non-null   float64
 5   mean_salary        1071 non-null   float64
 6   recycling_pct      1071 non-null   float64
 7   population_size    31 non-null     float64
 8   number_of_jobs     931 non-null    float64
 9   area_size          666 non-null    float64
 10  no_of_houses       666 non-null    float64
 11  borough_flag       1071 non-null   int64  
 12  mean_recycle_pct   903 non-null    float64
 13  extended_pop_size  651 non-null    object 
dtypes: float64(9), int64(1), object(4)
memory usage: 125.5+ KB


In [23]:
# Only 22 rows have NaN values in population_size column
# Remove commas and convert the "population_size" column to numeric
df['population_size'] = pd.to_numeric(df['population_size'].str.replace(',', ''), errors='coerce')

# Calculate the median of the "population_size" column
# median_population_size = df['population_size'].median()

# Fill NaN values in "population_size" column with the median value
# df['population_size'].fillna(median_population_size, inplace=True)

AttributeError: Can only use .str accessor with string values!

### Number of Jobs

In [ ]:
# Only 42 out of 140 NaN value remained
# Getting number of jobs data from csv
total_jobs = pd.read_csv("number_of_jobs.csv")

# List of columns to drop
columns_to_drop = ["name", "2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009", "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019", "2020", "2021"]

# Drop the specified columns from the DataFrame
total_jobs.drop(columns=columns_to_drop, inplace=True)

# Merging DataFrames based on the 'city' column (common key)
df = df.merge(total_jobs, on='area', how='left')

# Identify rows where "population_size" is NaN
mask = df['number_of_jobs'].isnull()

# Fill NaN values in "population_size" column with corresponding values from "extended_pop_size"
df.loc[mask, 'number_of_jobs'] = df.loc[mask, 'number_of_jobs_mean']

In [ ]:
# Remove commas and convert the "number_of_jobs" column to numeric
df['number_of_jobs'] = pd.to_numeric(df['number_of_jobs'].str.replace(',', ''), errors='coerce')

# Calculate the median of the "number_of_jobs" column
median_number_of_jobs = df['number_of_jobs'].median()

# Fill NaN values in "number_of_jobs" column with the median value
df['number_of_jobs'].fillna(median_number_of_jobs, inplace=True)

### Area Size

In [ ]:
# Only 105 out of 405 NaN value remained
# The correlation of 0.621 between population_size & area_size indicates a positive moderate-strong relationship.

# Rows where "area_size" column is not empty
non_empty_rows = df[df['area_size'].notna()]

# Only 42 out of 140 NaN value remained
# Getting number of jobs data from csv
total_hectar = pd.read_csv("total_hectar.csv")

# List of columns to drop
columns_to_drop = ["sample"]

# Drop the specified columns from the DataFrame
total_hectar.drop(columns=columns_to_drop, inplace=True)

# Merging DataFrames based on the 'city' column (common key)
df = df.merge(total_hectar, on='area', how='left')

# Identify rows where "population_size" is NaN
mask = df['area_size'].isnull()

# Fill NaN values in "population_size" column with corresponding values from "extended_pop_size"
df.loc[mask, 'area_size'] = df.loc[mask, 'total_hectar']

In [ ]:
# Remove commas and convert the "area_size" column to numeric
df['area_size'] = pd.to_numeric(df['area_size'].str.replace(',', ''), errors='coerce')

# Calculate the median of the "area_size" column
median_number_of_jobs = df['area_size'].median()

# Fill NaN values in "area_size" column with the median value
df['area_size'].fillna(median_number_of_jobs, inplace=True)

### Number of Houses

In [ ]:
# Calculating person per house
# df['persons_per_house'] = df['population_size'] / df['no_of_houses']

In [ ]:
# Fill NaN values in "no_of_houses" column with 0 - There is no number of house data for UK Regions
df["no_of_houses"].fillna(0, inplace=True)

### Dataframe Extraction

#### London Boroughs

In [ ]:
# The list of values you want to keep
london_values = ['city of london', 'barking and dagenham', 'barnet', 'bexley', 'brent', 'bromley', 'camden', 'croydon', 'ealing', 'enfield',
                  'greenwich', 'hackney', 'hammersmith and fulham', 'haringey', 'harrow', 'havering', 'hillingdon', 'hounslow', 'islington',
                  'kensington and chelsea', 'kingston upon thames', 'lambeth', 'lewisham', 'merton', 'newham', 'redbridge',
                  'richmond upon thames', 'southwark', 'sutton', 'tower hamlets', 'waltham forest', 'wandsworth', 'westminster']

# Filter the DataFrame to keep only rows with the desired values in the 'area' column
london_df = df[df['area'].isin(london_values)]

#### UK Regions

In [ ]:
# The list of values you want to keep
region_values = ['north east', 'north west', 'yorkshire and the humber', 'east midlands', 'west midlands', 'east',
                  'south east', 'south west', 'england', 'united kingdom', 'great britain', 'england and wales',
                  'northern ireland', 'scotland', 'wales', "inner london", "outer london", "london"]

# Filter the DataFrame to keep only rows with the desired values in the 'area' column
region_df = df[df['area'].isin(region_values)]

## Questions

### 1- Is there a correlation between the median salary and the life satisfaction for residents in London's boroughs?

- The correlation coefficient of -0.21936922410940793 suggests a weak negative correlation between "median_salary" and "life_satisfaction" in the dataset.

- Cost of living: London's high living costs might lead to higher salaries but potentially reduced life satisfaction due to financial stress.

- Work-life balance: Demanding jobs with higher salaries may lead to decreased life satisfaction if individuals experience limited leisure time or increased stress levels.

In [ ]:
# Set up seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create the histogram with customizations
plt.figure(figsize=(10, 6))
plt.hist(london_df["median_salary"], bins=20, color="#007acc", edgecolor="k", alpha=0.7)

# Add labels and title
plt.xlabel("Median Salary")
plt.ylabel("Frequency")
plt.title("Distribution of Median Salary of London Boroughs", fontsize=16)

# Add grid lines for better readability
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Remove the right and top spines for cleaner look
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Add a legend (optional) - if you have relevant data, like source or data year
#plt.legend(["2022 Data"])

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Set up seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create the histogram with customizations
plt.figure(figsize=(10, 6))
plt.hist(region_df["median_salary"], bins=20, color="#007acc", edgecolor="k", alpha=0.7)

# Add labels and title
plt.xlabel("Median Salary")
plt.ylabel("Frequency")
plt.title("Distribution of Median Salary of UK Regions", fontsize=16)

# Add grid lines for better readability
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Remove the right and top spines for cleaner look
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Add a legend (optional) - if you have relevant data, like source or data year
#plt.legend(["2022 Data"])

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Set up seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create the histogram with customizations
plt.figure(figsize=(10, 6))
plt.hist(london_df["life_satisfaction"], bins=20, color="#4caf50", edgecolor="k", alpha=0.7)

# Add labels and title
plt.xlabel("Life Satisfaction")
plt.ylabel("Frequency")
plt.title("Distribution of Life Satisfaction of London Boroughs", fontsize=16)

# Add grid lines for better readability
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Remove the right and top spines for cleaner look
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Add a legend (optional) - if you have relevant data, like source or data year
# plt.legend(["2022 Data"])

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Set up seaborn style for better aesthetics
sns.set(style="whitegrid")

# Create the histogram with customizations
plt.figure(figsize=(10, 6))
plt.hist(region_df["life_satisfaction"], bins=20, color="#4caf50", edgecolor="k", alpha=0.7)

# Add labels and title
plt.xlabel("Life Satisfaction")
plt.ylabel("Frequency")
plt.title("Distribution of Life Satisfaction of UK Regions", fontsize=16)

# Add grid lines for better readability
plt.grid(axis="y", linestyle="--", alpha=0.7)

# Remove the right and top spines for cleaner look
plt.gca().spines["right"].set_visible(False)
plt.gca().spines["top"].set_visible(False)

# Add a legend (optional) - if you have relevant data, like source or data year
# plt.legend(["2022 Data"])

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the Spearman's rank correlation coefficient between "median_salary" and "life_satisfaction"
correlation_coefficient = df["median_salary"].corr(df["life_satisfaction"], method="spearman")
correlation_coefficient

### 2- How does the recycling percentage vary across different boroughs of London? Are there any patterns or trends?

- Out of the 20 boroughs, the average recycling rate is above 26.9%.

- A correlation coefficient of 0.12 should be interpreted as a weak positive correlation between the mean_salary and recycle_pct_x being compared.

- The recycling rate increased until 2012, then entered a plateau phase, and in 2020, there was a drastic decline. The COVID-19 pandemic could have disrupted recycling operations and collection services. With lockdowns and social distancing measures, people might have been more reliant on single-use products, leading to increased waste generation.

- London 2012 won gold in the Environmental and Sustainability category at the 6th International Sports Event Management awards. However, after the success of the 2012 Olympics, priorities may have shifted, and there could have been a decrease in financial and political support for recycling projects. This reduction in investments may have affected the effectiveness and capacity of recycling activities.

In [ ]:
# Remove any rows with missing values (NaN)
mean_recycling_pct = mean_recycling_pct.dropna(subset=['recycling_pct'])

# Sort the data in descending order based on the mean of 'recycling_pct' column
mean_recycling_pct = mean_recycling_pct.sort_values(by='recycling_pct', ascending=False)

# Calculate the mean of 'recycling_pct' column
mean_value = mean_recycling_pct['recycling_pct'].mean()

In [ ]:
# Set up seaborn style for better aesthetics
sns.set(style="whitegrid")

# Plot the histogram
plt.figure(figsize=(20, 8))
bars = sns.barplot(x='area', y='recycling_pct', data=mean_recycling_pct, palette='viridis')

# Color bars above the mean line in green
for bar in bars.patches:
    if bar.get_height() > mean_value:
        bar.set_color('green')

# Add the mean line
plt.axhline(mean_value, color='red', linestyle='dashed', label='Mean')

plt.xlabel('Area', fontsize=16)
plt.ylabel('Recycling Percentage', fontsize=14)
plt.title('Recycling Percentages by Area', fontsize=16)
plt.xticks(rotation=90, fontsize=16)
plt.legend()
plt.tight_layout()

# Add a background color for the plot (optional)
plt.gca().set_facecolor('#f2f2f2')

# Remove the top and right spines for cleaner look
sns.despine()

plt.show()

In [ ]:
# Calculate the mean of the 'recycling_pct_y' column
# mean_recycling_pct_y = df['recycling_pct_x'].mean()

# Fill the NaN values in the 'recycling_pct_y' column with the calculated mean
# df['recycling_pct_x'].fillna(mean_recycling_pct_y, inplace=True)

# has_nan_values = df['recycling_pct_x'].isnull().any()
# has_nan_values

In [ ]:
# Group by 'date' and calculate the mean of 'recycling_pct_x'
grouped_df = df.groupby('date')['recycling_pct'].mean().reset_index()

# Create the trendline graph
plt.figure(figsize=(10, 6))
plt.plot(grouped_df['date'], grouped_df['recycling_pct'], marker='o', linestyle='-', color='b')
plt.xlabel('Date')
plt.ylabel('Recycling Percentage (%)')
plt.title('Trendline Graph of Recycling Percentage')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
# Calculate the correlation coefficient between 'mean_salary' and 'recycling_pct_x'
correlation_coefficient = df['mean_salary'].corr(df['recycling_pct'])
correlation_coefficient

In [ ]:
# Write London Boroughs data to CSV file
london_df.to_csv('london_data.csv', index=False)

In [ ]:
# Filter rows where mean_salary is below 20,504
filtered_df = london_df[london_df['mean_salary'] < 20504]
filtered_df